# Amazon Bedrock AgentCore Code Interpreter でのコマンド実行 - チュートリアル

このチュートリアルでは、Amazon Bedrock AgentCore Code Interpreter を使用してコマンド（シェルおよび AWS CLI）を実行する方法を説明します。AWS サービス、特に S3 操作に焦点を当てて対話します。以下の内容を説明します：

1. Code Interpreter の作成
2. Code Interpreter セッションの開始
3. コマンドの実行（シェルおよび AWS CLI）
5. S3 操作の実行（バケットの作成、オブジェクトのコピー、バケットオブジェクトの一覧表示）
6. クリーンアップ（セッションの停止と Code Interpreter の削除）



## 前提条件
- Bedrock AgentCore Code Interpreter へのアクセス権を持つ AWS アカウント
- Code Interpreter リソースを作成・管理するために必要な IAM 権限
- S3 操作を実行するために必要な IAM 権限
- 必要な Python パッケージのインストール（boto3 および bedrock-agentcore を含む）


## IAM 実行ロールには以下の IAM ポリシーをアタッチする必要があります

~~~ {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:CreateCodeInterpreter",
                "bedrock-agentcore:StartCodeInterpreterSession",
                "bedrock-agentcore:InvokeCodeInterpreter",
                "bedrock-agentcore:StopCodeInterpreterSession",
                "bedrock-agentcore:DeleteCodeInterpreter",
                "bedrock-agentcore:ListCodeInterpreters",
                "bedrock-agentcore:GetCodeInterpreter"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
        }
    ]
}

#### IAM 実行ロールには以下の信頼ポリシーもアタッチする必要があります。信頼ポリシーに bedrock-agentcore.amazonaws.com を含めることで、Bedrock Agent サービス自体がこの IAM ロールを引き受けて、ユーザーに代わってアクション（Amazon S3 など）を実行できるようになります。

```{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "AWS": "arn:aws:iam::<account_id>:root",
                "Service": [
                    "bedrock-agentcore.amazonaws.com"
                ]
            },
            "Action": "sts:AssumeRole",
            "Condition": {}
        }
    ]
}

さらに、このチュートリアルで説明する S3 操作を実行するには、IAM 実行ロールに AmazonS3FullAccess IAM ポリシーをアタッチする必要があります。

## 動作の仕組み

コード実行サンドボックスは、Code Interpreter、シェル、ファイルシステムを備えた隔離環境を作成することで、エージェントがユーザーのクエリを安全に処理できるようにします。Large Language Model がツール選択を支援した後、このセッション内でコードが実行され、その後ユーザーまたはエージェントに返されて統合されます。

![architecture local](code-interpreter.png)

## 1. 環境のセットアップ

まず、必要なライブラリをインポートします。

In [ ]:
!pip install --upgrade -r requirements.txt

In [ ]:
import json
import boto3
from bedrock_agentcore.tools.code_interpreter_client import code_session, CodeInterpreter
from bedrock_agentcore._utils import endpoints
import time
from typing import Dict, Any, List

## 2. 設定変数

Code Interpreter と S3 操作に必要な設定変数をセットアップします。また、他の AWS リソースにアクセスするために Code Interpreter に渡す IAM 実行ロールを提供します。このロールには、上記で説明した S3 権限が必要です。

In [ ]:
# Configuration setup
execution_role_arn = "<execution-role-arn>"
unique_bucket_name = f"amzn-bucket-{int(time.time())}"
s3_path = f"s3://{unique_bucket_name}"

region = "us-west-2"

#local file that we will upload to Code interpreter and then to an S3 bucket
local_file = "samples/stats.py"

## 3. エンドポイントのセットアップ

boto3 クライアントを作成するために、データプレーンとコントロールプレーンの両方のエンドポイントを設定する必要があります。

In [ ]:
# Configure endpoints
data_plane_endpoint = endpoints.get_data_plane_endpoint(region)
control_plane_endpoint = endpoints.get_control_plane_endpoint(region)

## 4. AWS クライアントの作成

コントロールプレーンとデータプレーン操作用の boto3 クライアントを初期化します。

In [ ]:
# Create boto3 clients
cp_client = boto3.client("bedrock-agentcore-control", 
                        region_name=region,
                        endpoint_url=control_plane_endpoint)

dp_client = boto3.client("bedrock-agentcore", 
                        region_name=region,
                        endpoint_url=data_plane_endpoint)

## 5. Code Interpreter の作成

特定の設定パラメータで Code Interpreter インスタンスを作成します。

Code Interpreter を設定する際、ネットワーク設定（Sandbox、Public、VPC）、依存関係の設定、セキュリティ設定、および Code Interpreter がアクセスできる AWS リソースを定義する IAM ランタイムロールによる権限を選択できます。

In [ ]:
# Create code interpreter
unique_name = f"s3InteractionEnv_{int(time.time())}"
interpreter_response = cp_client.create_code_interpreter(
    name=unique_name,
    description="S3 ファイル操作用環境",
    executionRoleArn=execution_role_arn,
    networkConfiguration={
        'networkMode': 'PUBLIC'
    }
)
interpreter_id = interpreter_response["codeInterpreterId"]
print(f"Code Interpreter を作成しました: {interpreter_id}")

## 6. セッションの開始

Code Interpreter 内でコードを実行するセッションを作成します。

In [ ]:
# Start session
session_response = dp_client.start_code_interpreter_session(
    codeInterpreterIdentifier=interpreter_id,
    name="s3InteractionSession",
    sessionTimeoutSeconds=900
)
session_id = session_response["sessionId"]
print(f"セッションを作成しました: {session_id}")

## 7. ツール実行用ヘルパー関数

Code Interpreter ツールの呼び出しを簡略化するユーティリティ関数を定義します。

In [ ]:
def call_tool(tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
    response = dp_client.invoke_code_interpreter(
        codeInterpreterIdentifier=interpreter_id,
        sessionId=session_id,
        name=tool_name,
        arguments=arguments
    )
    for event in response["stream"]:
        return json.dumps(event["result"], indent=2)

## 8. コード実行のテスト

### 8.1 シンプルな Hello World の例で Code Interpreter をテストします。

In [ ]:
# Test code execution
# S3 Operations
print("シェルコマンドを実行中 \n")
command_response = call_tool("executeCommand",
                              {"command": "echo 'Hello World'"})
print(f"コマンド結果: {command_response}")

# Parse and display results
command_results = json.loads(command_response)
print(command_results['structuredContent']['stdout'])

### 8.2 次に、PIP を使用してサンドボックスに boto3 をインストールします。

In [ ]:
# Test code execution
# S3 Operations
print("シェルコマンドを実行中 \n")
command_response = call_tool("executeCommand",
                              {"command": "pip install boto3"})

# Parse and display results
command_results = json.loads(command_response)
print(command_results['structuredContent']['stdout'])

## 9. コマンド実行によるファイル操作と S3 連携

#### 9.1 ローカルファイルをサンドボックスに書き込み

In [ ]:
# Write file to sandbox
print("サンドボックスにファイルを書き込み中")
try:
    with open(local_file, 'r', encoding='utf-8') as local_file_content:
        local_file_content = local_file_content.read()
except FileNotFoundError:
    print(f"エラー: ファイル '{local_file}' が見つかりませんでした。")
except Exception as e:
    print(f"エラーが発生しました: {e}")

files_to_create = [{
        "path": "stats.py",
        "text": local_file_content
}]
write_files_response = call_tool("writeFiles", {"content": files_to_create})
print(f"ファイル書き込み結果: {write_files_response}")

#### 9.2 Code Interpreter 経由で S3 バケットを作成

In [ ]:
# S3 Operations
print("\nS3 バケットを作成中")
create_s3_response = call_tool("executeCommand",
                              {"command": f"aws s3 mb {s3_path} --region {region}"})
print(f"作成結果: {create_s3_response}")

#### 9.3 コマンド実行により Code Interpreter からファイルを S3 バケット（上記で作成）にアップロード

In [ ]:
print("\nS3 にファイルをアップロード中")
upload_to_s3_response = call_tool("executeCommand",
                                 {"command": f"aws s3 cp {files_to_create[0]['path']} {s3_path}"})
print(f"アップロード結果: {upload_to_s3_response}")


#### 9.4 Code Interpreter でコマンドを実行して S3 バケットのファイルを一覧表示

In [ ]:
print("\nS3 内のファイルを一覧表示中")
list_s3_response = call_tool("executeCommand",
                            {"command": f"aws s3 ls {s3_path}"})
print(f"一覧表示結果: {list_s3_response}")

## 10. クリーンアップ

セッションを停止し、Interpreter を削除してリソースをクリーンアップします。

In [ ]:
# Cleanup
print("セッションと Interpreter をクリーンアップ中")
dp_client.stop_code_interpreter_session(
    codeInterpreterIdentifier=interpreter_id,
    sessionId=session_id
)
print("セッションが正常に停止しました")

cp_client.delete_code_interpreter(codeInterpreterId=interpreter_id)
print("Interpreter が正常に削除されました")